In [ ]:
#coding:utf-8
import numpy as np
import matplotlib.pyplot as plt
import gdal
import glob
from PyEMD import EEMD, EMD, Visualisation
from scipy import signal
from matplotlib.ticker import MultipleLocator,FixedLocator
import matplotlib.dates as mdates
import pandas as pd
from pandas.plotting import register_matplotlib_converters
from eofs.standard import Eof

In [ ]:
infile=r'F:\PDSI\data\pdsitif'
outfile=r'F:\PDSI\data\mkresult'
tifpath=infile+'\\'+'1901_1_1.tif'
dataset=gdal.Open(tifpath)
im_width = dataset.RasterXSize
im_height = dataset.RasterYSize
im_geotrans = dataset.GetGeoTransform()
im_proj = dataset.GetProjection()
del dataset
alltif=glob.glob(infile+'\\*.tif')
mydata=np.zeros((len(alltif),im_height,im_width))-999
for tif in range(len(alltif)):
    dataset=gdal.Open(alltif[tif])
    dataz = dataset.ReadAsArray(0,0,im_width,im_height)
    mydata[tif,:,:]=dataz
    del dataset

# Run the EEMD

In [ ]:
datas=[]
for j in range(1440):
    noe=mydata[j,:,:]
    noe=np.mean(noe[noe!=-9999])
    datas.append(noe)
T = np.arange(0, 1440,1)
S = datas
eemd = EEMD()
eemd.trials = 400
eemd.noise_seed(12345)
eemd.noise_width=0.2
E_IMFs = eemd.eemd(S, T, max_imf=-1)
eemd2 = EEMD()
eemd2.trials = 400
eemd2.separate_trends=True
eemd2.noise_seed(12345)
eemd2.noise_width=0.2
E_IMFs2 = eemd2.eemd(S, T, max_imf=-1)

imf=E_IMFs  #IMF result
res=E_IMFs2[-1] #RES reslut

# Run the EOF

In [ ]:
mydata[mydata==-9999]=np.nan
lat = np.arange(-90,90,0.5)
coslat = np.cos(np.deg2rad(lat))
wgts = np.sqrt(coslat)[..., np.newaxis]
solver = Eof(mydata, weights=wgts)
eof = solver.eofsAsCorrelation(neofs=7)
pc = solver.pcs(npcs=7, pcscaling=1)
var = solver.varianceFraction()
eigen_Values   = solver.eigenvalues()
percentContrib = eigen_Values * 100./np.sum(eigen_Values)
northTest =solver.northTest()
(np.abs(eigen_Values[1:]-eigen_Values[:-1])>northTest[:-1])[0:8]

# Calculate the annual drought characteristics

In [ ]:
def data_search(data, level):
    listt = []
    number=[]
    numbertem=[]
    temp = []
    for i in range(len(data)):
        if i < len(data)-1:
            if data[i] <= level:
                temp.append(data[i])
                numbertem.append(i)
            else:
                listt.append(temp)
                number.append(numbertem)
                temp = []
                numbertem=[]
        if i == len(data)-1:
            if data[i] <= level:
                temp.append(data[i])
                numbertem.append(i)
                listt.append(temp)
                number.append(numbertem)
            if data[i] > level:
                listt.append(temp)
                number.append(numbertem)
                temp = []
                numbertem=[]
    return [i for i in listt if i],[i for i in number if i]

In [ ]:
adsnewdata=np.zeros((int(mydata.shape[0]/12),im_height,im_width))-999
addnewdata=np.zeros((int(mydata.shape[0]/12),im_height,im_width))-999
adfnewdata=np.zeros((int(mydata.shape[0]/12),im_height,im_width))-999

ADS=np.zeros((im_height,im_width))-999
ADD=np.zeros((im_height,im_width))-999
ADF=np.zeros((im_height,im_width))-999
for j in range(mydata.shape[1]):
    for k in range(mydata.shape[2]):
        if np.where(mydata[:,j,k]==-9999)[0].shape[0]==0:
            for i in range(0,int(mydata.shape[0]/12)):
                nowdata=mydata[i*12:(i+1)*12,j,k]
                adsnewdata[i,j,k]=np.sum(nowdata[nowdata<=-3])
                addnewdata[i,j,k]=np.sum(len(nowdata[nowdata<=-3]))
                adfnewdata[i,j,k]=len(data_search(list(nowdata),-3))
            ADS[j,k]=np.mean(adsnewdata[:,j,k])
            ADD[j,k]=np.mean(addnewdata[:,j,k])
            ADF[j,k]=np.mean(adfnewdata[:,j,k])